# Tools to asses the performance of our functions
Main tools I use every day
* [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl) to benchmark the speed and memory allocations of our kernels
* Profilers to find bottlenecks (e.g. [ProfileCanvas.jl](https://github.com/pfitzseb/ProfileCanvas.jl))
* `@code_warntype` to check if our code is type stable
* [Cthulhu.jl](https://github.com/JuliaDebug/Cthulhu.jl)
Others
* [JET.jl](https://github.com/aviatesk/JET.jl)
* [Aqua.jl](https://github.com/JuliaTesting/Aqua.jl)

# BenchmarkTools.jl

In [1]:
using BenchmarkTools
f(x, n) = x^n
@btime f(1.2, 3.2)

  0.791 ns (0 allocations: 0 bytes)


1.7921732359737277

Be aware if you want to benchmark functions with non-literal arguments, you
need to use the dollar symbol:

In [2]:
x, n = 1.2, 3.2
@btime f($x, $n)

  10.803 ns (0 allocations: 0 bytes)


1.7921732359737277

# Type instabilities
A type instability when the compiler can't predict (infer) at compile time the types of a variable

In [3]:
function foo(x)
    v = 0.0
    for i in eachindex(x)
        v += x[i]
    end
    v
end

foo (generic function with 1 method)

If we benchmark the previous function with some funky argument, we will observe a non-expected behaviour

In [4]:
x = (1, 0.5, 34, 213.23f0)
@btime foo($x)

  63.903 ns (6 allocations: 224 bytes)


248.72999572753906

The best way to detect if and where we have type instabilities is with the `@code_warntype` macro

In [5]:
using InteractiveUtils
@code_warntype foo(x)

MethodInstance for Main.var"##557".foo(::Tuple{Int64, Float64, Int64, Float32})
  from foo(x) @ Main.var"##557" ~/Desktop/Seminars/JuliaMainz23/basic_perftools.ipynb:1
Arguments
  #self#::Core.Const(Main.var"##557".foo)
  x::Tuple{Int64, Float64, Int64, Float32}
Locals
  @_3::UNION{NOTHING, TUPLE{INT64, INT64}}
  v::Float64
  i::Int64
Body::Float64
1 ─       (v = 0.0)
│   %2  = Main.var"##557".eachindex(x)::Core.Const(Base.OneTo(4))
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3::Core.Const((1, 1)) === nothing)::Core.Const(false)
│   %5  = Base.not_int(%4)::Core.Const(true)
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = v::Float64
│   %11 = Base.getindex(x, i)::UNION{FLOAT32, FLOAT64, INT64}
│         (v = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       retur

If the argument is a homogeneous data container

In [6]:
x = (1.0, 0.5, 34.0, 213.23)
@code_warntype foo(x)

MethodInstance for Main.var"##557".foo(::NTuple{4, Float64})
  from foo(x) @ Main.var"##557" ~/Desktop/Seminars/JuliaMainz23/basic_perftools.ipynb:1
Arguments
  #self#::Core.Const(Main.var"##557".foo)
  x::NTuple{4, Float64}
Locals
  @_3::UNION{NOTHING, TUPLE{INT64, INT64}}
  v::Float64
  i::Int64
Body::Float64
1 ─       (v = 0.0)
│   %2  = Main.var"##557".eachindex(x)::Core.Const(Base.OneTo(4))
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3::Core.Const((1, 1)) === nothing)::Core.Const(false)
│   %5  = Base.not_int(%4)::Core.Const(true)
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│   %10 = v::Float64
│   %11 = Base.getindex(x, i)::Float64
│         (v = %10 + %11)
│         (@_3 = Base.iterate(%2, %9))
│   %14 = (@_3 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #4 if not %15
3 ─       goto #2
4 ┄       return v



# Profilers
Profilers can help you find bottlenecks in your code.

In [7]:
f1(x,y,n1,n2,a,b,c) = x^n1 * y^(n2-1) * exp((a + b) / c)
function f1()
    A = zeros(64, 64)
    for i in eachindex(A)
        x, y, n1, n2, a, b, c = rand(7)
        A[i] = f1(x,y,n1,n2,a,b,c)
    end
    return A
end
@btime f1();

  236.042 μs (4098 allocations: 480.05 KiB)


As you can see the performance is not great. Lets see where the bottleneck is with the profiler

In [8]:
using ProfileCanvas
ProfileCanvas.@profview for _ in 1:100 f1() end

ProfileCanvas.ProfileData(Dict{String, ProfileCanvas.ProfileFrame}("3" => ProfileCanvas.ProfileFrame("root", "", "", 0, 20, missing, 0x00, missing, ProfileCanvas.ProfileFrame[]), "4" => ProfileCanvas.ProfileFrame("root", "", "", 0, 20, missing, 0x00, missing, ProfileCanvas.ProfileFrame[]), "1" => ProfileCanvas.ProfileFrame("root", "", "", 0, 20, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#62", "task.jl", "./task.jl", 514, 16, missing, 0x10, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("macro expansion", "eval.jl", "/Users/albert/.vscode/extensions/julialang.language-julia-1.57.2/scripts/packages/VSCodeServer/src/eval.jl", 34, 16, missing, 0x01, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("invokelatest", "essentials.jl", "./essentials.jl", 816, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#invokelatest#2", "essentials.jl", "./essentials.jl", 819, 16, missing, 0x01, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#109", "repl.jl", "/Users/albert/.vscode/extensions/julialang.language-julia-1.57.2/scripts/packages/VSCodeServer/src/repl.jl", 193, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("with_logger", "logging.jl", "./logging.jl", 626, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("with_logstate", "logging.jl", "./logging.jl", 514, 16, missing, 0x01, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#110", "repl.jl", "/Users/albert/.vscode/extensions/julialang.language-julia-1.57.2/scripts/packages/VSCodeServer/src/repl.jl", 192, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("repleval", "repl.jl", "/Users/albert/.vscode/extensions/julialang.language-julia-1.57.2/scripts/packages/VSCodeServer/src/repl.jl", 229, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("eval", "Base.jl", "./Base.jl", 68, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("eval", "boot.jl", "./boot.jl", 370, 16, missing, 0x01, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("notebook", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 671, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("notebook", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 671, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#notebook#34", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 677, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("jupyter_notebook", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 740, 16, missing, 0x01, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("cd", "file.jl", "./file.jl", 112, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#38", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 741, 16, missing, 0x01, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("execute_notebook", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 754, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#execute_notebook#41", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 762, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("execute_block", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 844, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#execute_block#43", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 859, 16, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("capture", "IOCapture.jl", "/Users

In [9]:
function f2()
    A = zeros(64, 64)
    for i in eachindex(A)
        x, y, n1, n2, a, b, c = ntuple( _ -> rand(), Val(7))
        A[i] = f1(x, y, n1, n2, a, b, c)
    end
    return A
end
@btime f2();

  134.958 μs (2 allocations: 32.05 KiB)


In [10]:
ProfileCanvas.@profview for _ in 1:100 f2() end

ProfileCanvas.ProfileData(Dict{String, ProfileCanvas.ProfileFrame}("3" => ProfileCanvas.ProfileFrame("root", "", "", 0, 10, missing, 0x00, missing, ProfileCanvas.ProfileFrame[]), "4" => ProfileCanvas.ProfileFrame("root", "", "", 0, 10, missing, 0x00, missing, ProfileCanvas.ProfileFrame[]), "1" => ProfileCanvas.ProfileFrame("root", "", "", 0, 10, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#62", "task.jl", "./task.jl", 514, 6, missing, 0x10, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("macro expansion", "eval.jl", "/Users/albert/.vscode/extensions/julialang.language-julia-1.57.2/scripts/packages/VSCodeServer/src/eval.jl", 34, 6, missing, 0x01, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("invokelatest", "essentials.jl", "./essentials.jl", 816, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#invokelatest#2", "essentials.jl", "./essentials.jl", 819, 6, missing, 0x01, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#109", "repl.jl", "/Users/albert/.vscode/extensions/julialang.language-julia-1.57.2/scripts/packages/VSCodeServer/src/repl.jl", 193, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("with_logger", "logging.jl", "./logging.jl", 626, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("with_logstate", "logging.jl", "./logging.jl", 514, 6, missing, 0x01, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#110", "repl.jl", "/Users/albert/.vscode/extensions/julialang.language-julia-1.57.2/scripts/packages/VSCodeServer/src/repl.jl", 192, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("repleval", "repl.jl", "/Users/albert/.vscode/extensions/julialang.language-julia-1.57.2/scripts/packages/VSCodeServer/src/repl.jl", 229, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("eval", "Base.jl", "./Base.jl", 68, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("eval", "boot.jl", "./boot.jl", 370, 6, missing, 0x01, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("notebook", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 671, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("notebook", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 671, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#notebook#34", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 677, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("jupyter_notebook", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 740, 6, missing, 0x01, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("cd", "file.jl", "./file.jl", 112, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#38", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 741, 6, missing, 0x01, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("execute_notebook", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 754, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#execute_notebook#41", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 762, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("execute_block", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 844, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("#execute_block#43", "Literate.jl", "/Users/albert/.julia/packages/Literate/J03G7/src/Literate.jl", 859, 6, missing, 0x00, missing, ProfileCanvas.ProfileFrame[ProfileCanvas.ProfileFrame("capture", "IOCapture.jl", "/Users/albert/.julia/packag

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*